In [1]:
pip install surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 1.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357258 sha256=5e01385be3d7e5656c130644f2286c2550220518b799122c31e063de83c4c490
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [3]:
import pandas as pd
# import zipfile
import numpy as np
# import matplotlib.pyplot as plt
# from sklearn.metrics.pairwise import cosine_similarity
from surprise import Dataset, Reader, SVD,SVDpp,NMF, accuracy
from surprise.model_selection import train_test_split
from surprise.model_selection import cross_validate
from surprise.model_selection import GridSearchCV

**import data from files and split data on train and test**

In [38]:
# блок для ручного імпорту файлу
# movies = pd.DataFrame()
# ratings = pd.DataFrame()
# zip_file_path = 'ml-latest-small.zip'
# with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
#     with zip_ref.open('ml-latest-small/movies.csv') as f:
#         movies = pd.read_csv(f)
#     with zip_ref.open('ml-latest-small/ratings.csv') as f:
#         ratings = pd.read_csv(f)
#merge movies and user ratings into 1 DF
# data = pd.merge(ratings, movies, on= 'movieId')
# convert DF into surprise Dataset
# reader = Reader(rating_scale=(1,5))
# dataset = Dataset.load_from_df(data[['userId', 'movieId','rating']], reader)

dataset = Dataset.load_builtin("ml-100k") # імпорт даних засобами Surprise

**Підбір параметрів через GridSearchCV для SVD**


In [ ]:
n_factors = [50,70,100,120,150]
n_epochs = [20,40,60]
lr_all = [0.005, 0.0075,0.01]
reg_all = [0.02,0.05,0.1]

param_grid = {"n_factors": n_factors,"n_epochs": n_epochs, "lr_all": lr_all, "reg_all": reg_all}

gs = GridSearchCV(SVD, param_grid, measures=["rmse", "mae"], cv=3)
gs.fit(dataset)

print(gs.best_score["rmse"]) # best RMSE score
print(gs.best_params["rmse"]) # combination of parameters that gave the best RMSE score

# Result is:
# 0.9196658578999338
# {'n_factors': 150, 'n_epochs': 60, 'lr_all': 0.0075, 'reg_all': 0.1}

KeyboardInterrupt: 

**Підбір параметрів через GridSearchCV для SVD++**

In [ ]:
n_factors = [10,20,40]
n_epochs = [10,20,40]
lr_all = [0.005, 0.007]
reg_all = [0.02,0.05]

param_grid = {"n_factors": n_factors,"n_epochs": n_epochs, "lr_all": lr_all, "reg_all": reg_all}

gs = GridSearchCV(SVDpp, param_grid, measures=["rmse", "mae"], cv=3)
gs.fit(dataset)

print(gs.best_score["rmse"]) # best RMSE score
print(gs.best_params["rmse"]) # combination of parameters that gave the best RMSE score
# result is
# 0.9235333207149101
# {'n_factors': 20, 'n_epochs': 40, 'lr_all': 0.005, 'reg_all': 0.05}

0.9235333207149101
{'n_factors': 20, 'n_epochs': 40, 'lr_all': 0.005, 'reg_all': 0.05}


**Підбір параметрів через GridSearchCV для NMF**

In [ ]:
n_factors = [10,15,20]
n_epochs = [25,50,75]

param_grid = {"n_factors": n_factors,"n_epochs": n_epochs}

gs = GridSearchCV(NMF, param_grid, measures=["rmse", "mae"], cv=3, n_jobs=-1)
gs.fit(dataset)

print(gs.best_score["rmse"]) # best RMSE score
print(gs.best_params["rmse"]) # combination of parameters that gave the best RMSE score

# Result is:
# 0.9703281355610899
# {'n_factors': 10, 'n_epochs': 75}

0.9703281355610899
{'n_factors': 10, 'n_epochs': 75}


**перераховуємо на найкращій моделі - NMF з параметрами: 'n_factors': 10, 'n_epochs': 75**

In [6]:

trainset, testset = train_test_split(dataset, test_size=0.25, random_state = 42) # test set is made of 25% of the ratings.
model_svd = NMF(n_factors=10, n_epochs=75)
model_svd.fit(trainset) # Train the algorithm on the trainset, and predict ratings for the testset
predictions = model_svd.test(testset)
accuracy.rmse(predictions) # Then compute RMSE

# full cross validation
# cross_validate(model_svd, dataset, measures=["RMSE", "MAE"], cv=5, verbose=True)

# result is RMSE: 0.9639

RMSE: 0.9639


0.9638845863997593

**навчаємо модель з обраними параметрами на повних даних**

In [18]:
full_trainset = dataset.build_full_trainset()

model_svd = NMF(n_factors=10, n_epochs=75)
model_svd.fit(full_trainset)  # Train the algorithm on the trainset, and predict ratings for the testset

**формуємо рекомендацію**

In [37]:
df =  pd.DataFrame(dataset.__dict__['raw_ratings'], columns=['userId','movieId','rating','timestamp'])

user_id = '483' #робимо рекомендації для юзера 483
movies_to_recomend = df['movieId'].unique()
user_predictions = [(movie, model_svd.predict(user_id, movie).est) for movie in movies_to_recomend]
sorted_predictions = sorted(user_predictions, key=lambda x:x[1],reverse=True)

top_n = 5
print(f"Top {top_n} movie recomndation for user {user_id}:")
for i in range(top_n):
    print(f"MovieID: {sorted_predictions[i][0]}, predicted rating: {sorted_predictions[i][1]}")

# Result is:
# Top 5 movie recomndation for user 483:
# MovieID: 1642, predicted rating: 4.3237454747913775
# MovieID: 814, predicted rating: 4.225997619265404
# MovieID: 1449, predicted rating: 4.215746903081524
# MovieID: 1398, predicted rating: 4.164272828698931
# MovieID: 1125, predicted rating: 4.139227433454285

Top 5 movie recomndation for user 483:
MovieID: 1642, predicted rating: 4.3237454747913775
MovieID: 814, predicted rating: 4.225997619265404
MovieID: 1449, predicted rating: 4.215746903081524
MovieID: 1398, predicted rating: 4.164272828698931
MovieID: 1125, predicted rating: 4.139227433454285
